# Messaging services

- store and pull messages
- manage read receipts

In [ ]:
//| export

import { getDatabase } from "tinychat/db.ts";
import type { Database } from "tinychat/db.ts";
import { Record as Message } from "tinychat/api/types/chat/tinychat/core/message.ts";

export class Messaging {
  constructor(private db: Database) {}

  public receiveMessage(
    { m, uri, sender, time_us }: {
      m: Message;
      uri: string;
      sender: string;
      time_us: string;
    },
  ) {
    this.db.prepare(`
      INSERT INTO messages (uri, channel, server, text, sender, created_at, time_us) VALUES (
        :uri, :channel, :server, :text, :sender, :created_at, :time_us
      )`).run({
      uri,
      channel: m.channel,
      server: m.server,
      text: m.text,
      sender,
      created_at: m.createdAt,
      time_us: time_us,
    });
  }

  public static setupForTesting(): Messaging {
    const service = new Messaging(getDatabase({ reset: true }));
    // insert 2 test users
    [1, 2].forEach((i) => {
      service.db
        .prepare(
          `
        INSERT INTO users (did, handle, display_name, avatar, description) VALUES (
          :did, :handle, :display_name, :avatar, :description
        )
      `,
        )
        .run({
          did: `did:${i}`,
          handle: `user-${i}`,
          display_name: `User ${i}`,
          avatar: `http://google.com/avatar-${i}.jpeg`,
          description: `description ${i}`,
        });
    });
    // create test server
    service.db
      .prepare(
        `
      INSERT INTO servers (uri, name, creator) VALUES (
        :uri, :name, :creator
      )
    `,
      )
      .run({
        uri: "at://server-1",
        name: "Test Server",
        creator: "did:1",
      });

    // create memberships for both users
    [1, 2].forEach((i) => {
      service.db
        .prepare(
          `
        INSERT INTO server_memberships (user, server) VALUES (
          :user, :server
        )
      `,
        )
        .run({
          user: `did:${i}`,
          server: "at://server-1",
        });
    });

    // setup channels
    [1, 2].forEach((i) => {
      service.db
        .prepare(
          `
        INSERT INTO channels (uri, name, server) VALUES (
          :uri, :name, :server
        )
      `,
        )
        .run({
          uri: `at://channel-${i}`,
          name: `channel ${i}`,
          server: "at://server-1",
        });
    });
    return service;
  }

  public static init(): Messaging {
    return new Messaging(getDatabase({ reset: true }));
  }
}

In [ ]:
Messaging.init();

Messaging { db: SQLite3.Database { path: :memory: } }

In [ ]:
// import { j } from "@jurassic/jurassic";
// j.initialize("db.ipynb");

// await j``


In [ ]:
const get_time_us = (): string => `${new Date().getTime() * 1000}`;

Deno.test("test messaging service test setup", async (t) => {
  const messaging = Messaging.setupForTesting();

  await t.step("receive message", () => {
    messaging.receiveMessage({
      m: {
        channel: "at://channel-1",
        server: "at://server-1",
        text: "Hello, World!",
        createdAt: new Date().toISOString(),
      },
      uri: `at://message-${Math.random() * 1000}`,
      sender: "did:1",
      time_us: get_time_us(),
    });
  });
});